In [197]:
import sys
sys.path.append("../code")
import pandas as pd
import random
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [198]:
import blotto_utils as bu
from agent import Agent
import agent_utils as au

In [199]:
import scipy.optimize

In [200]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# 538 Data

In [7]:
weights = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

data_dir = '/Users/Alex/Documents/Alex/Yale/Fall 2019/CPSC_490/blotto/data/'

fivethirtyeight_data_dir = data_dir + 'fivethirtyeight/'

sorted(glob(fivethirtyeight_data_dir + '*'))

temp_df = pd.read_csv(glob(fivethirtyeight_data_dir + '*')[0])

new_column_names = list(map((lambda x : '_'.join(x.split(' ')).lower()), list(temp_df.columns)))
new_column_names[-1] = 'reason'

temp_df.rename(dict(zip(list(temp_df.columns), new_column_names)), axis=1, inplace=True)

temp_df['total_troops'] = temp_df['castle_1'] + \
                          temp_df['castle_2'] + \
                          temp_df['castle_3'] + \
                          temp_df['castle_4'] + \
                          temp_df['castle_5'] + \
                          temp_df['castle_6'] + \
                          temp_df['castle_7'] + \
                          temp_df['castle_8'] + \
                          temp_df['castle_9'] + \
                          temp_df['castle_10']

full_strats_df = temp_df[temp_df.total_troops == 100].copy()

full_strats_list = full_strats_df.to_dict('records')

simp_strats_list = [[int(x[1]) for x in list(y.items())[:10]] for y in full_strats_list]

## Divide into two players

In [8]:
random.seed(0)

In [9]:
a_strat_init = random.sample(simp_strats_list, 100)
b_strat_init = random.sample(simp_strats_list, 100)

In [10]:
len(a_strat_init), len(b_strat_init)

(100, 100)

# Agent

In [66]:
random.seed(0)

In [67]:
temp_noise = bu.generate_noise_arrays(1)

In [68]:
def evolve(a_strats, b_strats, generations, noise):
    for _ in range(generations):
        a_results, b_results = bu.eval_strats(a_strats, b_strats, weights)

        a_fittest = sorted(a_results, key=(lambda x: (x[1], -(x[0][-1]))), reverse=True)
        b_fittest = sorted(b_results, key=(lambda x: (x[1], -(x[0][-1]))), reverse=True)

        a_next_gen = []
        b_next_gen = []
        for i in range(14):
            for j in range(i+1, 15):
                a_next_gen.append(bu.mutate_noise(bu.make_integer(bu.cross([a_fittest[i][0], a_fittest[j][0]]), seed=(i+j+i*j)), temp_noise, seed=(i+j)))
                b_next_gen.append(bu.mutate_noise(bu.make_integer(bu.cross([b_fittest[i][0], b_fittest[j][0]]), seed=(i+j+i*j)), temp_noise, seed=(i+j)))

        a_strats = a_next_gen
        b_strats = b_next_gen
        
    return a_fittest, b_fittest, a_strats, b_strats

In [69]:
a_strat_init = random.sample(simp_strats_list, 100)
b_strat_init = random.sample(simp_strats_list, 100)

a_strats = a_strat_init
b_strats = b_strat_init

In [70]:
temp_a_strats, temp_b_strats = a_strats, b_strats

In [71]:
best_a, best_b = [], []

In [72]:
for i in range(200):
#     print(i)
    temp_a_fittest, temp_b_fittest, temp_a_strats, temp_b_strats = evolve(temp_a_strats, temp_b_strats, 1, temp_noise)
    best_a.append(temp_a_fittest[0][0])
    best_b.append(temp_b_fittest[0][0])
#     print('\n'.join(map(str,temp_a_fittest[:5])))

In [73]:
len(best_a), len(best_b)

(200, 200)

# Linear Programming Testing (Nash)

input are two sets of strategies -> probabilities associated with each for Nash

## testing

In [138]:
probs_a, probs_b = bu.blotto_nash(a_strat_init, b_strat_init)

In [139]:
sum(probs_a)

1.0000000000000002

In [140]:
sum(probs_b)

1.0304870000000002

In [167]:
def normalize_probabilities(probs):
    probs = np.array(probs)
    probs /= probs.sum()
    return probs

In [168]:
probs_a_norm = normalize_probabilities(probs_a)
probs_b_norm = normalize_probabilities(probs_b)

In [175]:
print(sum(probs_a_norm))
print(sum(probs_b_norm))

1.0000000000000002
1.0


In [271]:
agent_a = Agent(a_strat_init, list(probs_a_norm))
agent_b = Agent(b_strat_init, list(probs_b_norm))

In [274]:
au.play_game(agent_a, agent_b, weights, weights, trials=1000) / 1000 # b is the better agent on average

0.473

In [275]:
au.play_game(agent_a, simp_strats_list, weights, weights, trials=len(simp_strats_list)) / len(simp_strats_list)

0.6168327796234773

In [276]:
au.play_game(agent_b, simp_strats_list, weights, weights, trials=len(simp_strats_list)) / len(simp_strats_list)

0.5060908084163898